# Credit Spread Prediction – Data Collection
This notebook downloads and cleans macro-financial data from FRED and Yahoo Finance.

In [2]:
!mkdir MF703_CreditSpreadProject
%cd MF703_CreditSpreadProject

!pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm fredapi yfinance shap statsmodels



/content/MF703_CreditSpreadProject


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from fredapi import Fred

In [11]:
fred = Fred(api_key="2dee14a9f6222157bf414d8fd93746dc")

# Corporate yields
aaa_yield = fred.get_series('AAA')     # Moody's AAA Corporate Bond Yield
baa_yield = fred.get_series('BAA')     # Moody's BAA Corporate Bond Yield

# Treasury yield (10-year constant maturity)
treasury_10y = fred.get_series('GS10') # 10-year Treasury

# Monthly versions
aaa_m = aaa_yield.resample('M').last()
baa_m = baa_yield.resample('M').last()
t10_m = treasury_10y.resample('M').last()

# Credit spreads vs Treasuries
spread_AAA_10Y = (aaa_m - t10_m).rename('AAA_10Y_Spread')
spread_BAA_10Y = (baa_m - t10_m).rename('BAA_10Y_Spread')

# Optional: AA corporate yield
try:
    aa_yield = fred.get_series('DAAA').resample('M').last()
    spread_AA_10Y = (aa_yield - t10_m).rename('AA_10Y_Spread')
except:
    pass

# Spread between credit qualities (BAA - AAA)
credit_curve_slope = (spread_BAA_10Y - spread_AAA_10Y).rename('BAA_minus_AAA')

# Combine spreads
spread_df = pd.concat([
    spread_AAA_10Y,
    spread_BAA_10Y,
    credit_curve_slope
], axis=1).dropna()

spread_df.head(), spread_df.shape



/tmp/ipython-input-3434508562.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  aaa_m = aaa_yield.resample('M').last()
/tmp/ipython-input-3434508562.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  baa_m = baa_yield.resample('M').last()
/tmp/ipython-input-3434508562.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  t10_m = treasury_10y.resample('M').last()
/tmp/ipython-input-3434508562.py:21: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  aa_yield = fred.get_series('DAAA').resample('M').last()


(            AAA_10Y_Spread  BAA_10Y_Spread  BAA_minus_AAA
 1953-04-30            0.40            0.82           0.42
 1953-05-31            0.29            0.73           0.44
 1953-06-30            0.29            0.75           0.46
 1953-07-31            0.35            0.93           0.58
 1953-08-31            0.29            0.90           0.61,
 (871, 3))

In [10]:
df = pd.read_csv('credit_spread_dataset.csv', index_col=0, parse_dates=True)
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 102 entries, 2000-01-31 to 2025-04-30
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Spread        102 non-null    float64
 1   VIX           102 non-null    float64
 2   GDP           102 non-null    float64
 3   Unemployment  102 non-null    float64
 4   YieldCurve    102 non-null    float64
dtypes: float64(5)
memory usage: 4.8 KB


,Spread,VIX,GDP,Unemployment,YieldCurve
count,102.000000,102.000000,102.000000,102.000000,102.000000
mean,1.021961,20.084902,17635.671676,5.702941,1.091275
std,0.411796,7.933551,5515.639091,2.050185,0.987462
min,0.550000,10.180000,10002.179000,3.400000,-0.910000
25%,0.780000,14.600000,13782.615000,4.200000,0.205000
50%,0.925000,18.470000,16369.980000,5.050000,1.055000
75%,1.137500,23.515000,20888.082750,6.350000,1.917500
max,3.090000,59.889999,30485.729000,14.800000,2.840000
